In [1]:
!pip install transformers==4.38.0
!pip install datasets
!pip install torch==2.0.1+cu110
!pip install peft


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 26.9 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.2
    Uninstalling transformers-4.40.2:
      Successfully uninstalled transformers-4.40.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 14.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggi

lora

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, Trainer, TrainingArguments
from datasets import load_dataset, load_metric
from peft import get_peft_model, LoraConfig
import torch
import numpy as np

# Tokenize data
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

# Define evaluation metric
def compute_metrics(eval_preds):
    metric = load_metric("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Define tokenizer and model and data collator
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2).cuda() if torch.cuda.is_available() else AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Load dataset
data = load_dataset("glue", "mrpc")
train_data, valid_data, test_data = data["train"], data["validation"], data["test"]

# Preprocess data
train_dataset = train_data.map(tokenize_function, batched=True, remove_columns=['idx','sentence1','sentence2'])
valid_dataset = valid_data.map(tokenize_function, batched=True, remove_columns=['idx','sentence1','sentence2'])
test_dataset = test_data.map(tokenize_function, batched=True, remove_columns=['idx','sentence1','sentence2'])

# Define PEFT configuration
peft_config = LoraConfig(task_type="SEQ_CLS",
                        r=4,
                        lora_alpha=32,
                        lora_dropout=0.01,
                        target_modules = ['query'])

# Apply PEFT to the model
model = get_peft_model(model, peft_config)

# define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=1e-3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True,
    seed=42
)

# creater trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# train model
trainer.train()

# test model
trainer.predict(test_dataset)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.581800,0.445014,0.794118,0.856655
2,0.456200,0.418157,0.823529,0.875862
3,0.365100,0.451905,0.791667,0.839925
4,0.295000,0.446213,0.813725,0.866197
5,0.230700,0.484915,0.813725,0.864286


<ipython-input-2-5ae0b8f45822>:13: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("glue", "mrpc")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/glue/glue.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/glue/glue.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possi

/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/glue/glue.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


PredictionOutput(predictions=array([[-1.4706208 ,  1.8487391 ],
       [-2.1946828 ,  2.5006037 ],
       [-1.4935042 ,  1.8378971 ],
       ...,
       [-0.22088751,  0.6208285 ],
       [-1.1958159 ,  1.5719521 ],
       [-1.9279646 ,  2.348506  ]], dtype=float32), label_ids=array([1, 1, 1, ..., 0, 1, 1]), metrics={'test_loss': 0.45459237694740295, 'test_accuracy': 0.7982608695652174, 'test_f1': 0.8556016597510374, 'test_runtime': 8.3826, 'test_samples_per_second': 205.783, 'test_steps_per_second': 12.884})

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, Trainer, TrainingArguments
from datasets import load_dataset, load_metric
from peft import get_peft_model, LoraConfig
import torch
import numpy as np

# Tokenize data
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

# Define evaluation metric
def compute_metrics(eval_preds):
    metric = load_metric("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Define tokenizer and model and data collator
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2).cuda() if torch.cuda.is_available() else AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Load dataset
data = load_dataset("glue", "mrpc")
train_data, valid_data, test_data = data["train"], data["validation"], data["test"]

# Preprocess data
train_dataset = train_data.map(tokenize_function, batched=True, remove_columns=['idx','sentence1','sentence2'])
valid_dataset = valid_data.map(tokenize_function, batched=True, remove_columns=['idx','sentence1','sentence2'])
test_dataset = test_data.map(tokenize_function, batched=True, remove_columns=['idx','sentence1','sentence2'])

# Define PEFT configuration
peft_config = LoraConfig(task_type="SEQ_CLS",
                        r=1,
                        lora_alpha=32,
                        lora_dropout=0.01,
                        target_modules = ['query'])

# Apply PEFT to the model
model = get_peft_model(model, peft_config)

# define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=1e-3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True,
    seed=42
)

# creater trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# train model
trainer.train()

# test model
trainer.predict(test_dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.587700,0.574838,0.737745,0.837139
2,0.478700,0.489890,0.784314,0.853333
3,0.401700,0.483105,0.791667,0.847397
4,0.344100,0.473211,0.799020,0.860544
5,0.296200,0.512279,0.794118,0.855670


/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/glue/glue.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
Checkpoint destination directory ./results/checkpoint-230 already exists and is non-empty. Saving will proceed but saved results may be invalid.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load

/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/glue/glue.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


PredictionOutput(predictions=array([[-1.1552765,  2.0703356],
       [-1.3743078,  2.6123998],
       [-1.3719616,  2.4705675],
       ...,
       [-0.528451 ,  1.1064227],
       [-1.4771526,  2.6125555],
       [-0.8292141,  1.3849913]], dtype=float32), label_ids=array([1, 1, 1, ..., 0, 1, 1]), metrics={'test_loss': 0.47299104928970337, 'test_accuracy': 0.7913043478260869, 'test_f1': 0.8491198658843252, 'test_runtime': 8.6015, 'test_samples_per_second': 200.546, 'test_steps_per_second': 12.556})

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, Trainer, TrainingArguments
from datasets import load_dataset, load_metric
from peft import get_peft_model, LoraConfig
import torch
import numpy as np

# Tokenize data
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

# Define evaluation metric
def compute_metrics(eval_preds):
    metric = load_metric("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Define tokenizer and model and data collator
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2).cuda() if torch.cuda.is_available() else AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Load dataset
data = load_dataset("glue", "mrpc")
train_data, valid_data, test_data = data["train"], data["validation"], data["test"]

# Preprocess data
train_dataset = train_data.map(tokenize_function, batched=True, remove_columns=['idx','sentence1','sentence2'])
valid_dataset = valid_data.map(tokenize_function, batched=True, remove_columns=['idx','sentence1','sentence2'])
test_dataset = test_data.map(tokenize_function, batched=True, remove_columns=['idx','sentence1','sentence2'])

# Define PEFT configuration
peft_config = LoraConfig(task_type="SEQ_CLS",
                        r=16,
                        lora_alpha=32,
                        lora_dropout=0.01,
                        target_modules = ['query'])

# Apply PEFT to the model
model = get_peft_model(model, peft_config)

# define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=1e-3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True,
    seed=42
)

# creater trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# train model
trainer.train()

# test model
trainer.predict(test_dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.572500,0.458926,0.806373,0.865417
2,0.438300,0.439847,0.821078,0.873484
3,0.329600,0.459596,0.816176,0.865350
4,0.250300,0.502741,0.816176,0.868651
5,0.177800,0.582295,0.816176,0.868190


/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/glue/glue.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
Checkpoint destination directory ./results/checkpoint-230 already exists and is non-empty. Saving will proceed but saved results may be invalid.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load

/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/glue/glue.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


PredictionOutput(predictions=array([[-1.2456176,  2.3050508],
       [-0.6305416,  1.622322 ],
       [-1.5832111,  2.832495 ],
       ...,
       [-0.8647527,  1.6541317],
       [-1.1738269,  2.124179 ],
       [-0.9691645,  1.8867148]], dtype=float32), label_ids=array([1, 1, 1, ..., 0, 1, 1]), metrics={'test_loss': 0.43008407950401306, 'test_accuracy': 0.8057971014492754, 'test_f1': 0.8588284871470713, 'test_runtime': 8.5231, 'test_samples_per_second': 202.392, 'test_steps_per_second': 12.671})

bitfit

In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, Trainer, TrainingArguments
from datasets import load_dataset, load_metric
from peft import get_peft_model, LoraConfig
import torch
import numpy as np

# Tokenize data
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

# Define evaluation metric
def compute_metrics(eval_preds):
    metric = load_metric("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Define tokenizer and model and data collator
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2).cuda() if torch.cuda.is_available() else AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Load dataset
data = load_dataset("glue", "mrpc")
train_data, valid_data, test_data = data["train"], data["validation"], data["test"]

# Preprocess data
train_dataset = train_data.map(tokenize_function, batched=True, remove_columns=['idx','sentence1','sentence2'])
valid_dataset = valid_data.map(tokenize_function, batched=True, remove_columns=['idx','sentence1','sentence2'])
test_dataset = test_data.map(tokenize_function, batched=True, remove_columns=['idx','sentence1','sentence2'])

#Bitfit
num_param = 0
for name, param in model.named_parameters():
    if "bias" not in name:
        param.requires_grad = False
    else:
        num_param += param.numel()

print("trainable params: {} || all params: {} || trainable%: {}".format(num_param, sum(param.numel() for param in model.parameters()), num_param / sum(param.numel() for param in model.parameters())))

# define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=1e-3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True,
    seed=42
)

# creater trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# train model
trainer.train()

# test model
trainer.predict(test_dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.605500,0.503456,0.740196,0.835913
2,0.516200,0.442990,0.808824,0.863636
3,0.470300,0.421365,0.803922,0.848485
4,0.441600,0.397526,0.830882,0.877442
5,0.409800,0.395470,0.825980,0.874780


<ipython-input-2-1c1f5248f685>:13: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("glue", "mrpc")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/glue/glue.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/glue/glue.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/glue/glue.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major rele

/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/glue/glue.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


PredictionOutput(predictions=array([[-0.82849497,  0.83913445],
       [ 0.05809403, -0.10933584],
       [-1.6932485 ,  1.4456315 ],
       ...,
       [-1.5826427 ,  1.4211832 ],
       [-1.630496  ,  1.4264804 ],
       [-1.4698936 ,  1.4734664 ]], dtype=float32), label_ids=array([1, 1, 1, ..., 0, 1, 1]), metrics={'test_loss': 0.4299057722091675, 'test_accuracy': 0.8081159420289855, 'test_f1': 0.8593285167870803, 'test_runtime': 8.1103, 'test_samples_per_second': 212.691, 'test_steps_per_second': 13.316})